In [1]:
using Pkg; Pkg.update

using ParallelTemperingMonteCarlo,BenchmarkTools,StaticArrays

Looking into the implementation of the EAM. A semi-two-body potential 

In [2]:
ico_13  = [[-0.0000000049,       -0.0000000044,       -0.0000000033],
[-0.0000007312,       -0.0000000014,        0.6554619119],
[0.1811648930,      -0.5575692094,        0.2931316798],
[-0.4742970242,       -0.3445967289,        0.2931309525],
[-0.4742970303,        0.3445967144,        0.2931309494],
[0.1811648830,        0.5575692066,        0.2931316748],
[0.5862626299,        0.0000000022,        0.2931321262],
[-0.1811648928,       -0.5575692153,       -0.2931316813],
[-0.5862626397,       -0.0000000109,       -0.2931321327],
[-0.1811649028,        0.5575692007,       -0.2931316863],
[0.4742970144,        0.3445967202,       -0.2931309590],
[0.4742970205,       -0.3445967231,       -0.2931309559],
[0.0000007214,       -0.0000000073,       -0.6554619185]]
ico_55 = [[0.0000006584,       -0.0000019175,        0.0000000505],
[-0.0000005810,       -0.0000004871,        0.6678432175],
[0.1845874248,       -0.5681026047,        0.2986701538],
[-0.4832557457,       -0.3511072166,        0.2986684497],
[-0.4832557570,        0.3511046452,        0.2986669456],
[0.1845874064,        0.5681000550,        0.2986677202],
[0.5973371920,       -0.0000012681,        0.2986697030],
[-0.1845860897,       -0.5681038901,       -0.2986676192],
[-0.5973358752,       -0.0000025669,       -0.2986696020],
[-0.1845861081,        0.5680987696,       -0.2986700528],
[0.4832570624,        0.3511033815,       -0.2986683486],
[0.4832570738,       -0.3511084803,       -0.2986668445],
[0.0000018978,       -0.0000033480,       -0.6678431165],
[-0.0000017969,        0.0000009162,        1.3230014650],
[0.1871182835,       -0.5758942175,        0.9797717078],
[-0.4898861924,       -0.3559221410,       0.9797699802],
[-0.4898862039,        0.3559224872,        0.9797684555],
[0.1871182648,        0.5758945856,        0.9797692407],
[0.6055300485,        0.0000001908,        0.9797712507],
[0.7926501864,       -0.5758950093,        0.6055339635],
[0.3656681761,       -1.1254128670,        0.5916673591],
[-0.3027660545,       -0.9318173412,        0.6055326929],
[-0.9573332453,       -0.6955436707,        0.5916639831],
[-0.9797705418,       -0.0000006364,        0.6055294407],
[-0.9573332679,        0.6955423392,        0.5916610035],
[-0.3027660847,        0.9318160902,        0.6055287012],
[0.3656681396,        1.1254115783,        0.5916625380],
[0.7926501677,        0.5758937939,        0.6055314964],
[1.1833279992,       -0.0000006311,        0.5916664660],
[0.6770051458,       -0.9318186223,        0.0000033028],
[0.0000006771,       -1.1517907207,        0.0000025175],
[-0.6770037988,       -0.9318186442,        0.0000007900],
[-1.0954155825,       -0.3559242494,       -0.0000012200],
[-1.0954155940,        0.3559203788,       -0.0000027447],
[-0.6770038290,        0.9318147872,       -0.0000032017],
[0.0000006397,        1.1517868856,       -0.0000024165],
[0.6770051155,        0.9318148091,       -0.0000006889],
[1.0954168993,        0.3559204143,        0.0000013211],
[1.0954169108,       -0.3559242139,        0.0000028458],
[0.3027674014,       -0.9318199253,       -0.6055286002],
[-0.3656668229,       -1.1254154134,       -0.5916624370],
[-0.7926488510,       -0.5758976290,       -0.6055313954],
[-1.1833266824,       -0.0000032040,       -0.5916663649],
[-0.7926488697,        0.5758911742,       -0.6055338624],
[-0.3656668594,        1.1254090319,       -0.5916672580],
[0.3027673712,        0.9318135061,       -0.6055325919],
[0.9573345621,        0.6955398357,       -0.5916638820],
[0.9797718586,       -0.0000031986,       -0.6055293396],
[0.9573345846,       -0.6955461743,       -0.5916609025],
[-0.1871169480,       -0.5758984207,       -0.9797691397],
[-0.6055287318,       -0.0000040259,       -0.9797711497],
[-0.1871169667,        0.5758903824,       -0.9797716067],
[0.4898875091,        0.3559183059,       -0.9797698792],
[0.4898875207,       -0.3559263223,       -0.9797683545],
[0.0000031136,       -0.0000047513,       -1.3230013639]]

nmtobohr = 18.8973
copperconstant = 0.36258*nmtobohr
pos_cu13 = copperconstant*ico_13
pos_cu55 = copperconstant*ico_55
AtoBohr = 1.8897259886
bc_cu13 = SphericalBC(radius=8*AtoBohr)
start_config = Config(pos_cu13, bc_cu13)

Config{13, SphericalBC{Float64}, Float64}(SVector{3, Float64}[[-3.35737368666e-8, -3.01478453496e-8, -2.2610884012200003e-8], [-5.0100237544608e-6, -9.5924962476e-9, 4.491082807389623], [1.2413025402138254, -3.8203432492477134, 2.0084746703815606], [-3.2497803034902475, -2.3611020206489175, 2.00846968707976], [-3.249780345286124, 2.3611019212980637, 2.0084696658392325], [1.241302471695995, 3.8203432300627207, 2.0084746361226458], [4.016944341017041, 1.50739226748e-8, 2.008477729017507], [-1.2413025388434689, -3.8203432896732337, -2.0084746806592353], [-4.016944408164515, -7.468443507060001e-8, -2.008477773554097], [-1.2413026073612992, 3.8203431896372013, -2.0084747149181505], [3.2497802363427737, 2.361101961038405, -2.00846973161635], [3.2497802781386502, -2.361101980908576, -2.0084697103758224], [4.9428762807276005e-6, -5.0018016148200005e-8, -4.491082852611391]], SphericalBC{Float64}(228.54811596737582))

Generating the necessary dist2_matrix

In [3]:
dis2mat = [distance2(a,b) for a in pos_cu13, b in pos_cu13]
dist2mat_55 = [distance2(a,b) for a in pos_cu55, b in pos_cu55]

55×55 Matrix{Float64}:
  0.0      20.939    20.939    20.939   …   62.2806   62.2806   82.1728
 20.939     0.0      23.1496   23.1496     144.657   144.657   186.072
 20.939    23.1496    0.0      23.1496     121.19     83.2196  140.213
 20.939    23.1496   23.1496    0.0        144.657   121.19    140.213
 20.939    23.1496   60.6064   23.1496     121.19    144.657   140.213
 20.939    23.1496   60.6064   60.6064  …   83.2196  121.19    140.213
 20.939    23.1496   23.1496   60.6064      83.2196   83.2196  140.213
 20.939    60.6064   23.1496   23.1496      83.2196   45.249    66.0106
 20.939    60.6064   60.6064   23.1496      83.2196   83.2196   66.0106
 20.939    60.6064   83.756    60.6064      45.249    83.2196   66.0106
  ⋮                                     ⋱                      
 82.1728  140.213   140.213   186.072       22.7447   69.2332   90.848
 62.2806  121.19     83.2196  144.657       23.7891   23.7891   69.2332
 82.1728  140.213    66.0106  140.213       69.2332   22

The mathematical form of the EAM is:

$$
E = \frac{1}{2}\sum_{i,j :i \neq j}\epsilon\left(\frac{a}{r_{ij}}\right)^n - \epsilon C \sum_{i}\left[\sum_{j\neq i} \left(\frac{a}{r_{ij}}\right)^m \right]^{1/2}
$$

We have two terms, one similar to the LJ repulsion term, one a density term $F(\rho _i)$.

We can consider that $\frac{\epsilon a^n}{2}$ and $\epsilon C a^{m/2} $ can be taken out of the sum. Thus a single sum represents contracting a vector in the dist matrix. 

In [4]:
# struct EmbeddedAtomPotential <: AbstractDimerPotential
#     n::Float64
#     m::Float64
#     ean::Float64
#     eCam::Float64
# end
# function EmbeddedAtomPotential(n,m,ϵ,C,a)
#     epsan = ϵ * a^n / 2
#     epsCam = ϵ * C * a^(m/2)
#     return EmbeddedAtomPotential(n,m,epsan,epsCam)
# end

In [5]:
evtohartree = 0.0367493

n = 8.482
m = 4.692
ϵ = evtohartree*0.0370
a = 0.25*nmtobohr
C = 27.561


copperpot = EmbeddedAtomPotential(n,m,ϵ,C,a)

suttonchenpot = EmbeddedAtomPotential(9.0,6.0,0.0126*evtohartree,39.432,0.3612*nmtobohr)

copperpot2 = EmbeddedAtomPotential(9.05 , 5.005 , 0.0225*evtohartree,33.17,0.327*nmtobohr)

EmbeddedAtomPotential(9.05, 5.005, 5949.446479587037, 2.6153101861477057)

In [6]:
invrexp(r2,expterm) = ifelse(r2 != 0 , 1/sqrt(r2)^expterm, 0. )
calcvec(distvec,expterm) = sum(invrexp.(distvec,Ref(expterm)))

function energyatomi(distvec,pot::EmbeddedAtomPotential) 
    Fterm = pot.eCam*sqrt(calcvec(distvec,pot.m))
    phiterm = pot.ean*calcvec(distvec,pot.n)
    return phiterm - Fterm
end
function energytotal(distmatrix,pot::EmbeddedAtomPotential)
    total_energy = 0.
    for distrow in eachrow(distmatrix)
        total_energy += energyatomi(distrow,pot)
    end
    return total_energy 
end
E_vec = [energyatomi(distrow,copperpot) for distrow in eachrow(dis2mat)]
sum(E_vec)

E_vec2 = [energyatomi(distrow,suttonchenpot) for distrow in eachrow(dis2mat)]
sum(E_vec2)

-1.3002351409450545

In [7]:
energytotal(dis2mat,suttonchenpot)

-1.3002351409450545

In [8]:
@benchmark energytotal($dist2mat_55,$suttonchenpot)

BenchmarkTools.Trial: 10000 samples with 1 evaluation.
 Range (min … max):  34.807 μs …  3.749 ms  ┊ GC (min … max): 0.00% … 98.44%
 Time  (median):     45.353 μs              ┊ GC (median):    0.00%
 Time  (mean ± σ):   44.583 μs ± 79.395 μs  ┊ GC (mean ± σ):  3.95% ±  2.20%

   ▁█                     ▂▃▁                                  
  ███▇▇▃▃▂▂▂▂▂▂▂▂▁▂▁▁▂▂▃▄████▇▆▅▅▄▃▃▃▃▃▃▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂ ▃
  34.8 μs         Histogram: frequency by time        60.7 μs <

 Memory estimate: 53.28 KiB, allocs estimate: 110.

# An alternative approach

Considering that the $\rho_i$ term depends on the square of sums, we can't simply update the total energy by considering the change to a single $r_{ij}$ on the global energy.

Rather than calculating $E_i$ and being unable to _separate_ the terms, we can store the $\phi_i,\rho_i$ terms. This may result in slower calculation of the global terms, but it may speed up the calculation of $E_{new}$ at each step.

In [9]:
function invrexp(r2,n,m)
    if r2 != 0.
        r_term = 1/sqrt(r2)
        return r_term^n , r_term^m
    else
        return 0. , 0.
    end    
end

function calc_components(eatomvec,distancevec,n,m)
    for dist in distancevec
        eatomvec .+= invrexp(dist,n,m)
    end
    return eatomvec
end
function calc_all_components(dist2_mat,n,m)
    n_atoms = size(dist2_mat)[1]
    component_vector = zeros(n_atoms,2)
    for row_index in 1:n_atoms
        component_vector[row_index,:] = calc_components(component_vector[row_index,:],dist2_mat[row_index,:],n,m)
    end
    return component_vector
end
function calc_energies_from_components(component_vector,ean,ecam)
    return sum(ean.*component_vector[:,1] .- ecam*sqrt.(component_vector[:,2]))
end
function calc_total_energy(dist2_mat,pot::EmbeddedAtomPotential)
    componentvec = calc_all_components(dist2_mat,pot.n,pot.m)
    return componentvec,calc_energies_from_components(componentvec,pot.ean,pot.eCam) 
end

calc_total_energy (generic function with 1 method)

In [10]:
phirho,energytot = calc_total_energy(dist2mat_55,suttonchenpot)

([1.3923704520825358e-5 0.0014529219103246252; 1.1152248517093746e-5 0.0012372868606771957; … ; 6.176322918802852e-6 0.0007295761752523578; 5.340590516223844e-6 0.0005944969082196186], -6.119246949010012)

In [11]:
@benchmark calc_total_energy($dist2mat_55,$copperpot)

BenchmarkTools.Trial: 10000 samples with 1 evaluation.
 Range (min … max):  118.840 μs …  5.181 ms  ┊ GC (min … max): 0.00% … 97.21%
 Time  (median):     126.821 μs              ┊ GC (median):    0.00%
 Time  (mean ± σ):   128.636 μs ± 86.515 μs  ┊ GC (mean ± σ):  1.16% ±  1.68%

       █                                                        
  ▁▁▁▁▂█▄▃▂▂▆▃▂▂▅█▇▆▄▄▅▄▄▃▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁ ▂
  119 μs          Histogram: frequency by time          150 μs <

 Memory estimate: 34.34 KiB, allocs estimate: 116.

In [12]:
function update_components(componentvec,atomindex,old_r2_vec,new_r2_vec,n,m)
    new_component_vec= copy(componentvec)

    for j_index in eachindex(new_r2_vec)
        j_term = invrexp(new_r2_vec[j_index],n,m) .- invrexp(old_r2_vec[j_index],n,m)
        new_component_vec[j_index,:] .+= j_term 
        new_component_vec[atomindex,:] .+= j_term 
    end
    return new_component_vec
end

update_components (generic function with 1 method)

In [13]:
testindex,test_disp = 3,[0.5,0.5,-0.5]
test_pos = pos_cu55[testindex] + test_disp

3-element Vector{Float64}:
  1.764752985534391
 -3.3925157884546686
  1.5464230925690106

In [14]:
newdisvector = [distance2(test_pos,b) for b in pos_cu55]
newdisvector[testindex] = 0.

testenergyd2mat = copy(dist2mat_55)
testenergyd2mat[3,:] = newdisvector
testenergyd2mat[:,3] = newdisvector

newphirho,newentot = calc_total_energy(testenergyd2mat,suttonchenpot)

new_component_vector = update_components(phirho,3,dist2mat_55[3,:],newdisvector,9.,6.)
enew = calc_energies_from_components(new_component_vector,suttonchenpot.ean,suttonchenpot.eCam)

-6.0788077775169675

In [15]:
function calc_new_energy(componentvec,atomindex,old_r2,new_r2,pot::EmbeddedAtomPotential)
    new_component_vector = update_components(componentvec,atomindex,old_r2,new_r2,pot.n,pot.m)
    newenergy = calc_energies_from_components(new_component_vector,pot.ean,pot.eCam)

    return new_component_vector , newenergy 
end

calc_new_energy (generic function with 1 method)

In [16]:
calc_new_energy(phirho,3,dist2mat_55[3,:],newdisvector,suttonchenpot)[2]

-6.0788077775169675

In [17]:
@benchmark calc_new_energy($phirho,$3,$dist2mat_55[3,:],$newdisvector,$suttonchenpot)

BenchmarkTools.Trial: 10000 samples with 8 evaluations.
 Range (min … max):  3.014 μs … 669.771 μs  ┊ GC (min … max):  0.00% … 98.61%
 Time  (median):     3.581 μs               ┊ GC (median):     0.00%
 Time  (mean ± σ):   5.007 μs ±  20.748 μs  ┊ GC (mean ± σ):  12.99% ±  3.12%

    ██                                                         
  ▁▅███▄▃▂▂▂▂▂▂▁▂▁▁▁▁▁▁▁▁▂▁▁▁▁▂▃▅▅▅▅▄▄▃▃▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁ ▂
  3.01 μs         Histogram: frequency by time        7.57 μs <

 Memory estimate: 12.48 KiB, allocs estimate: 117.

# Finishing up

With the majority of the functions complete, it's just a matter of including them in the main code. Specifically how this is done is never the most fun challenge, but something to work on nevertheless.

In [18]:
get_energy(dist2mat_55,suttonchenpot)

([1.3923704520825358e-5 0.0014529219103246252; 1.1152248517093746e-5 0.0012372868606771957; … ; 6.176322918802852e-6 0.0007295761752523578; 5.340590516223844e-6 0.0005944969082196186], -6.119246949010012)

# Incorporating this  into MCRun

With the energy functions complete, it's time to migrate this to mc_cycles. Because we have the additional output, we need to tailor the move function as well. I think the next goal is to create a new way to handle these data (change vectors within the states struct to prevent constant redefinition)

In [19]:
n_atoms = 55

ti = 550
tf = 900

n_traj = 20

temp = TempGrid{n_traj}(ti,tf)
mc_cycles = 100
mc_sample = 1
displ_atom = 0.1 # Angstrom
n_adjust = 100
max_displ_atom = [0.1*sqrt(displ_atom*temp.t_grid[i]) for i in 1:n_traj]
mc_params = MCParams(mc_cycles, n_traj, n_atoms, mc_sample = mc_sample, n_adjust = n_adjust)
move_strat = MoveStrategy(atom_moves = n_atoms)  
ensemble = NVT(n_atoms)
bc_cu55 = SphericalBC(radius=14*AtoBohr)
start_config = Config(pos_cu55, bc_cu55)
n_bin = 100
mc_states = [MCState(temp.t_grid[i], temp.beta_grid[i], start_config, suttonchenpot) for i in 1:n_traj]
results = Output{Float64}(n_bin; en_min = mc_states[1].en_tot)

Output{Float64}(100, 0.0, 0.0, Float64[], Float64[], Float64[], Vector{Float64}[], Matrix{Float64}[], Vector{Float64}[], Float64[], Float64[], Float64[], Float64[])

In [20]:
ptmc_run!((mc_states, move_strat, mc_params, suttonchenpot, ensemble, results); save=false)

UndefVarError: UndefVarError: i_atom not defined

In [21]:
distance2(mc_states[1].config.pos[1],mc_states[1].config.pos[2])

20.93901164147001